# **Analytical Comparison Between CEO Concerns**


> ## _Author: Ronald Washington III_ 

> > Our society is currently run by some of the worlds largest and smartest technology
companies. Where each of these companies have a vision and impact on the world due to the
views of their CEO. But how do the views between these CEOs differ? What are the topics that
these CEOs are constantly concerned with and are expressing their opinions? Within this study
we shall collect tweets from selected CEOs, determine if we are able to identify the tweet’s
originator correctly, and extract the topic of each tweet to have a collection of concerns that each
CEO has. From there the analysis can be furthered by calculating the distance between the issues
that the CEOs talk about and have an understanding of how closely related their concerns are.

> > Conducting our analysis through the assistance of an classification algorithm will aid in
our research concerning the topics that CEOs primarily tweet about. The specific algorithm that
will be performed in this research will be Support Vector Machines, which is known to be useful
in the classification of images, bioinformatics, and text categorization. Support Vector Machines
(SVMs) are going to be useful determining which CEOs a tweet belong to, and we will be
utilizing Latent Dirichlet Allocation to extract prominent topics among the classified tweets.
Through the usage of both these techniques, we strive to accomplish our goal of classifying
tweets and finding the common topics discussed among CEOs. By the end of this research we
shall attempt to distinguish the difference between topics of CEOs, the similarities/differences,
and the number of times a specific topic is tweeted.


### Imported Libraries 

In [1]:
import keys as k
import pandas as pd
import webbrowser
import requests
import bs4
import importlib
import tweepy
import json 
from requests_oauthlib import OAuth1Session
importlib.reload(k)

<module 'keys' from 'C:\\Users\\Ron\\Desktop\\CS401\\Final_Project\\CEO_Classifier_Extractor\\keys.py'>

### Preparing/Gathering Authorization for Twitter Access

In [2]:
twitter = k.twitter
client_key = twitter["client_key"]
client_secret = twitter["client_secret"]
resource_owner_key = twitter['resource_owner_key']
resource_owner_secret = twitter['resource_owner_secret']
protected_url = 'https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name=jlist'

session = OAuth1Session(client_key,
                        client_secret=client_secret,
                        resource_owner_key=resource_owner_key,
                        resource_owner_secret= resource_owner_secret)

consumer_key = twitter["client_key"]
consumer_secret = twitter["client_secret"]
access_token = twitter['resource_owner_key']
access_token_secret =  twitter['resource_owner_secret']
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

# Developing a list of CEOs

> Within this study we will be utlizing a list of CEOs provided by Valeria Maltoni (@ConversationAge), who created a member group/list of CEOs who use Twitter. 
- https://twitter.com/ConversationAge/lists/ceos/members?lang=en

In [3]:

group_name = 'ceos'
owner = 'ConversationAge'


def get_list_members(api, owner, group_name):
    members = []
    # without this you only get the first 20 list members
    for person in tweepy.Cursor(api.list_members, owner, group_name).items():
        number_tweets = person.statuses_count
        if number_tweets>=1000:
            members.append(person)
    # create a list containing all usernames
    return [ m.screen_name for m in members ]

In [4]:
len(get_list_members(api,owner,group_name))

655

In [5]:
CEO_list = pd.DataFrame()
CEO_list['CEO_Twitter_Accounts'] = get_list_members(api,owner,group_name)
CEO_list.head(5)

,CEO_Twitter_Accounts
0,DrJudyMonroe
1,JillStorey2020
2,co_opcloud
3,PaulPolman
4,KenLingad


In [6]:
len(CEO_list)

655

In [7]:
PrivateList = []
TweetCountList = []
import time
t0 = time.time()

for i in range(len(CEO_list)):
    number_tweets = api.get_user(CEO_list['CEO_Twitter_Accounts'][i]).statuses_count
    if number_tweets>=1000:
        x = api.get_user(CEO_list['CEO_Twitter_Accounts'][i])
        TweetCountList.append(number_tweets)
        PrivateList.append(x.protected)
    #print(x.screen_name,x.protected)
t1 = time.time()
total = ((t1-t0)/60)
print("Time to Run: ", total)
import collections
counter=collections.Counter(PrivateList)
print("Count of Non-Private Versus Private Twitter Accounts: ", counter)

Time to Run:  16.74964051246643
Count of Non-Private Versus Private Twitter Accounts:  Counter({False: 650, True: 5})


In [8]:
se = pd.Series(PrivateList)
te = pd.Series(TweetCountList)
CEO_list['Private_Indicator'] = se.values
CEO_list['Tweet_Count'] = te.values

In [9]:
CEO_list[25:30]

,CEO_Twitter_Accounts,Private_Indicator,Tweet_Count
25,techUKCEO,False,3259
26,morganberman,False,1548
27,jane_knows,False,1372
28,limouris,False,2185
29,Monique_Villa,False,1532


In [10]:
CEO_list_cleaned = pd.DataFrame()
CEO_list_cleaned = CEO_list
CEO_list_cleaned = CEO_list_cleaned[CEO_list['Private_Indicator']!=True]

In [11]:
len(CEO_list_cleaned)

650

# Gathering Tweets

In [129]:
import tweepy
import json
import time
def tweetCollector(dataframe):
    """
    Creates a CSV for every CEO's tweets 
    """
    function_start = time.time()
    list_tweets = []
    name = []
    user_name = []
    count = 0
    for i in dataframe:
        
        for page in tweepy.Cursor(api.user_timeline, screen_name='@'+i,include_rts = False, tweet_mode = 'extended').pages(100):
            for status in page:

                list_tweets.append(status._json['full_text'])
                name.append(status._json['user']['name'])
                user_name.append(status._json['user']['screen_name'])
                DataSet = pd.DataFrame()

                DataSet['Tweet'] = [tweet for tweet in list_tweets]
                DataSet['CEO_Full_Name'] = [tweet for tweet in name]
                DataSet['CEO_User_Name'] = [tweet for tweet in user_name]
        count +=1
        print("CEO #: ", count, "CEO Name: ",i)
    DataSet.to_csv("Collection_CEO_TWEETS.csv")
    
    function_end = time.time()
    total_function_time = ((function_end-function_start)/60)
    print("Time to Complete: ", total_function_time)
    
    return DataSet


#tweetCollector(CEO_list_cleaned['CEO_Twitter_Accounts'][:50])
#tweetCollector(CEO_list_cleaned['CEO_Twitter_Accounts'][50:99])
#tweetCollector(CEO_list_cleaned['CEO_Twitter_Accounts'][99:149])
#tweetCollector(CEO_list_cleaned['CEO_Twitter_Accounts'][149:199])
tweetCollector(CEO_list_cleaned['CEO_Twitter_Accounts'][199:249])


CEO #:  1 CEO Name:  nickbayer
CEO #:  2 CEO Name:  JakeTDunlap
CEO #:  3 CEO Name:  NedBreslin
CEO #:  4 CEO Name:  denisehterry
CEO #:  5 CEO Name:  dbiemesderfer
CEO #:  6 CEO Name:  jmlazard
CEO #:  7 CEO Name:  DMiliband
CEO #:  8 CEO Name:  AmFamJack
CEO #:  9 CEO Name:  howiegoldfarb
CEO #:  10 CEO Name:  rickperreault
CEO #:  11 CEO Name:  mtbert
CEO #:  12 CEO Name:  JLNY
CEO #:  13 CEO Name:  UnmistakableCEO
CEO #:  14 CEO Name:  thebask
CEO #:  15 CEO Name:  betashop
CEO #:  16 CEO Name:  andreagoulet
CEO #:  17 CEO Name:  Bill_George
CEO #:  18 CEO Name:  dessaigne
CEO #:  19 CEO Name:  andykurtzig
CEO #:  20 CEO Name:  MichaelDell
CEO #:  21 CEO Name:  jeremyheimans
CEO #:  22 CEO Name:  kristofdewulf
CEO #:  23 CEO Name:  khadim5
CEO #:  24 CEO Name:  RehmanSid
CEO #:  25 CEO Name:  BreveMente
CEO #:  26 CEO Name:  meparker7
CEO #:  27 CEO Name:  RobertLReynolds
CEO #:  28 CEO Name:  drorpockard
CEO #:  29 CEO Name:  adamquinton
CEO #:  30 CEO Name:  natedapore
CEO #:  3

,Tweet,CEO_Full_Name,CEO_User_Name
0,Further proof that businesses do well by doing...,Nick Bayer,nickbayer
1,"Grateful for many things, including the podcas...",Nick Bayer,nickbayer
2,Good morning @philly311 - these containers (20...,Nick Bayer,nickbayer
3,"My feelings exactly, @cindeezly https://t.co/I...",Nick Bayer,nickbayer
4,@PhillyGrub @BigRubeHarley @Saxbys Agreed!!,Nick Bayer,nickbayer


- Figure out how to get all the tweets for each CEO
- Put all tweets together into a large corpus (Each CEO gets ~3200 tweets and here are 902 Tweets so 288k tweets roughly)
- Perform Support Vector Machine Algorithm to classify which tweet belongs to which CEO

In [33]:
df = pd.read_csv("Collection_1_CEO_TWEETS.csv",encoding = "latin1")
df.head()

,Unnamed: 0,Tweet,CEO_Full_Name,CEO_User_Name
0,0,Your gift to @CDCFound supports our collaborat...,"Judy Monroe, MD",DrJudyMonroe
1,1,By giving to the causes you care about on #Giv...,"Judy Monroe, MD",DrJudyMonroe
2,2,"As you give thanks this holiday season, please...","Judy Monroe, MD",DrJudyMonroe
3,3,"This #Thanksgiving, were #thankful to you for...","Judy Monroe, MD",DrJudyMonroe
4,4,Enjoyed attending the #KoplanLecture on Friday...,"Judy Monroe, MD",DrJudyMonroe


##### Here I have found that I am unable to grab the same number of tweets per person for some reason. So in the rest of this research I will be only caring about individuals that I was able to successfully grab more than 1000 tweets. 

In [127]:
df.CEO_User_Name.value_counts().head()

DelMonteLouis1    2000
co_opcloud        1999
JohnLegere        1935
CarrieMantha      1868
scottpetinga      1838
Name: CEO_User_Name, dtype: int64

In [104]:
# x = df.groupby('CEO_Full_Name')['Tweet'].apply(list)
# j = pd.DataFrame(x)
# j
# j.reset_index()
# j.T
# xx = j.T
# print(xx['AQ']['Tweet'])

CEO_Full_Name,AQ,Adam Greenbaum,Andrew Limouris,Anthony Ferrier,Balaji S. Srinivasan,Ben Watts,Bill Nuti,Brandon Edwards,Brian A Russell,Brook Parker Bello PH.D,...,Rachel Gerrol,Scott Petinga,Sujay R Jadhav,Tamara McCleary,Terri Seese,Tony Duffin,Victoria Espinel,Yael Lehmann,Zack Kanter,Zen Yinger
Tweet,[@JamaMusse - do you have an email address I c...,[This is the stupidest shit I have ever seen. ...,[Need professional gift ideas? https://t.co/RQ...,"[Great practical session from @scottdanthony, ...",[Slack is much better if you turn off all soun...,[5 ways to make sure your company values are t...,"[After January 14, 2020, no security updates, ...","[McKinsey Insights: New demand, new markets: W...",[@chuckwoolery @RobertM97254372 The stacked Ju...,[Victim of Hillsong Church founder's father sa...,...,[Join Juno and get 30% off all rides for two w...,[Fashion is a three trillion dollar industry w...,"[https://t.co/4SBUY5qQZT, https://t.co/gZp3APF...","[I recently interviewed Dr. May Wang, #CTO @Zi...",[@kelly_clarkson and @PearlDrumCorp Love it! H...,[?@AnaLiffey? Drug Project Helped Over 400 Peo...,"[@DataStax @BigData_Review Congrats!, Using @a...",[Follow along at noon today ET! Honored to be ...,"[@sachinrekhi Which talk was this in?, I guess...","[Yes, all time consuming buying tasks are cond..."
